# Transfer learning on mouse data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Defines imports
import ipykernel
import os
import sys
module_path = os.path.abspath(os.path.join('/home/gustavo/Gdrive/Stanford/Lab/ODF_prediction/cleaned_scripts/utilities')) # or the path to your source code
sys.path.insert(0, module_path)

import numpy as np
import keras
import tensorflow as tf
import tensorflow_probability as tfp

import matplotlib.pyplot as plt
import os
import random
import pickle

from tensorflow.keras.optimizers import Adam, SGD
from sklearn.preprocessing import MinMaxScaler
import scipy.stats as scis
import scipy.io as scio


from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model

import model_functions as mfunc
from model_functions import final_layer_gamma
from model_functions import negative_log_likelihood_loss_gamma

import evaluation

In [3]:
# Defines constants
_DATASET_LIST = ['dog','dog_panda','dog_sealion_panda','dog_sealion','dog_goldfish_hotpot_panda_sealion_terrier', 'goldfish_hotpot_panda_sealion_terrier']
_DATASET_FOLDER = '/home/gustavo/Gdrive/Stanford/Lab/ODF_prediction/datasets/experimental'
_MODEL_FOLDER = '/home/gustavo/Gdrive/Stanford/Lab/ODF_prediction/cleaned_scripts/pretrained_models'
_RESULTS_FOLDER = '/home/gustavo/Gdrive/Stanford/Lab/ODF_prediction/cleaned_scripts/experimental_results'
_TL_MODEL_FOLDER = '/home/gustavo/Gdrive/Stanford/Lab/ODF_prediction/cleaned_scripts/tf_models'
_NUM_RUNS = 20
_LEARNING_RATE = 0.0015
_NUM_EPOCHS = 2500
_BATCH_SIZE = 16
_LAYERS_FREEZE = [0,1,2,3,6,7,8,9]

### Load dataset

In [4]:
dataset_name = 'mouse_3regions'
file_name = os.path.join(_DATASET_FOLDER,f'{dataset_name}.pkl')
with open(file_name, 'rb') as file:
    data_save = pickle.load(file)

# Unpacks the data
train_data = data_save['train']
val_data = data_save['val']
x_train = train_data['x_scaled']
diameter_samples_train = train_data['diameter_samples']
gratio_samples_train = train_data['gratio_samples']
x_val = val_data['x_scaled']
diameter_samples_val = val_data['diameter_samples']
gratio_samples_val = val_data['gratio_samples']

# Defines and compiles network
num_features = x_train.shape[1]

## Fine tuning

In [5]:
custom_objects = {"final_layer_gamma": final_layer_gamma, 
                  "negative_log_likelihood_loss_gamma": negative_log_likelihood_loss_gamma}

for dataset_name in _DATASET_LIST:
    # Load model
    path_to_model = os.path.join(_MODEL_FOLDER,'pretrain_'+dataset_name)
    for iteration in range(_NUM_RUNS):
        print(f'Processing {dataset_name}, run {iteration}')
        with keras.utils.custom_object_scope(custom_objects):
            model = keras.models.load_model(path_to_model)
        for i,layer in enumerate(model.layers):
            if i in _LAYERS_FREEZE:
                layer.trainable = False
            else:
                layer.trainable = True
        opt = Adam(learning_rate = _LEARNING_RATE)
        run_name = f'experimental_{dataset_name}_{iteration}'
        # We already used validation for hyperparameter selection, so we combine train an validation for final retrain
        history = model.fit(np.concatenate([x_train,x_val]), 
                            {'axon_output': np.concatenate([diameter_samples_train,
                                                            diameter_samples_val]),
                            'gratio_output': np.concatenate([gratio_samples_train,
                                                             gratio_samples_val])},
                            epochs=_NUM_EPOCHS, shuffle=True, batch_size=_BATCH_SIZE, verbose=0)
        name_loss_figure = os.path.join(_RESULTS_FOLDER,f'loss_{run_name}.png')
        # evaluation.plot_losses(history,name_loss_figure)
        # Save model
        save_name = os.path.join(_TL_MODEL_FOLDER,dataset_name,run_name)
        model.save(save_name)

Processing dog, run 0
INFO:tensorflow:Assets written to: /home/gustavo/Gdrive/Stanford/Lab/ODF_prediction/cleaned_scripts/tf_models/dog/experimental_dog_0/assets
Processing dog, run 1
INFO:tensorflow:Assets written to: /home/gustavo/Gdrive/Stanford/Lab/ODF_prediction/cleaned_scripts/tf_models/dog/experimental_dog_1/assets
Processing dog, run 2
INFO:tensorflow:Assets written to: /home/gustavo/Gdrive/Stanford/Lab/ODF_prediction/cleaned_scripts/tf_models/dog/experimental_dog_2/assets
Processing dog, run 3
INFO:tensorflow:Assets written to: /home/gustavo/Gdrive/Stanford/Lab/ODF_prediction/cleaned_scripts/tf_models/dog/experimental_dog_3/assets
Processing dog, run 4
INFO:tensorflow:Assets written to: /home/gustavo/Gdrive/Stanford/Lab/ODF_prediction/cleaned_scripts/tf_models/dog/experimental_dog_4/assets
Processing dog, run 5
INFO:tensorflow:Assets written to: /home/gustavo/Gdrive/Stanford/Lab/ODF_prediction/cleaned_scripts/tf_models/dog/experimental_dog_5/assets
Processing dog, run 6
INFO:t